# UK - Anbindung Paid Themen 

* Um was handelt es sich hier  (Kurzbeschreibung Inhalt):
- Die Paid Data Transformatation von LinkedIn


---
* QUELLEN:  
- datif_pz_uk_{}.03_transformed.linkedin_paid_daily
- datif_pz_uk_{}.03_transformed.linkedin_paid_total


* ZIEL:  
- datif_pz_uk_{}.03_transformed.linkedin_paid_organic_daily
- datif_pz_uk_{}.03_transformed.linkedin_paid_organic_total


---
* Versionen (aktuelle immer oben):
- 17.09.2025 Justin Stange-Heiduk: init


# 1. Initialnotebooks & Libraries

In [0]:
%run ../../common/nb_init

In [0]:
target_schema_name = "03_transformed"
target_path = "funnel"

## 03-transformed

### Daily

In [0]:
df_daily = spark.sql(f"""
                    CREATE OR REPLACE VIEW datif_pz_uk_{env}.03_transformed.linkedin_paid_organic_daily AS
                    SELECT
                        CASE
                            WHEN lo.PostID is NULL Then "Paid"
                            WHEN lp.AdID is NULL Then "Organic"
                            ELSE "Organic&Paid"
                        END AS Source,
                        COALESCE(lo.PostID,'Existiert nicht') AS OrganicID,
                        COALESCE(lo.Date, lp.Date, 'Existiert nicht') AS Date,
                        COALESCE(lo.CreatedDate, 'Existiert nicht') AS CreatedDate,
                        COALESCE(lo.PostContent, lp.Text,  'Existiert nicht') AS Title,
                        COALESCE(lo.PostURL, 'Existiert nicht') AS URL,
                        COALESCE(lo.ContentType, 'Existiert nicht') AS PostType,
                        COALESCE(lo.Impressions, 0) AS Organic_Impressions,
                        COALESCE(lo.Engagements, 0) AS Organic_Engagements,
                        COALESCE(round(lo.WeightedEngagements,2), 0) AS Organic_WeightedEngagements,
                        COALESCE(lo.Shares, 0) AS Organic_Shares,
                        COALESCE(lo.Comments, 0) AS Organic_Comments,
                        COALESCE(lo.Likes, 0) AS Organic_Likes,
                        COALESCE(lo.Clicks, 0) AS Organic_Clicks,
                        COALESCE(lp.AdID, 'Existiert nicht') AS AdID,
                        COALESCE(lp.CampaignName, 'Existiert nicht') AS CampaignName,
                        COALESCE(lp.AdSetName, 'Existiert nicht') AS AdSetName,
                        COALESCE(lp.AdName, 'Existiert nicht') AS AdName,
                        COALESCE(round(lp.Amount_Spend,2), 0) AS Amount_Spend,
                        COALESCE(lp.Impressions, 0) AS Paid_Impressions,
                        COALESCE(lp.Engagements, 0) AS Paid_Engagements,
                        COALESCE(round(lp.WeightedEngagements,2), 0) AS Paid_WeightedEngagements,
                        COALESCE(lp.Post_Shares, 0) AS Paid_Shares,
                        COALESCE(lp.Comments, 0) AS Paid_Comments,
                        COALESCE(lp.Likes, 0) AS Paid_Likes,
                        COALESCE(lp.Clicks, 0) AS Paid_Clicks,
                        COALESCE(Organic_Impressions + Paid_Impressions, 0) AS All_Impressions,
                        COALESCE(Organic_Engagements + Paid_Engagements, 0) AS All_Engagements,
                        COALESCE(round(Organic_WeightedEngagements + Paid_WeightedEngagements,2), 0) AS All_WeightedEngagements,
                        COALESCE(Organic_Shares + Paid_Shares, 0) AS All_Shares,
                        COALESCE(Organic_Comments + Paid_Comments, 0) AS All_Comments,
                        COALESCE(Organic_Likes + Paid_Likes, 0) AS All_Likes,
                        COALESCE(Organic_Clicks + Paid_Clicks, 0) AS All_Clicks
                    FROM datif_pz_uk_{env}.03_transformed.linkedin_organic_daily_view AS lo
                    FULL OUTER JOIN datif_pz_uk_{env}.03_transformed.linkedin_paid_daily AS lp
                        ON TRIM(LOWER(lo.PostContent)) = TRIM(LOWER(lp.Text))
                        AND CAST(lo.Date AS DATE) = CAST(lp.Date AS DATE)
                    """)

### Total

In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW datif_pz_uk_{env}.03_transformed.linkedin_paid_organic_total AS
SELECT
    CASE
        WHEN lo.PostID IS NULL THEN "Paid"
        WHEN lp.ID IS NULL THEN "Organic"
        ELSE "Organic&Paid"
    END AS Source,
    COALESCE(lo.PostID, 'Existiert nicht') AS OrganicID,
    COALESCE(lo.CreatedDate, 'Existiert nicht') AS Organic_CreatedDate,
    COALESCE(lp.CreatedDate, 'Existiert nicht') AS Paid_CreatedDate,
    COALESCE(lo.PostContent, lp.Text, 'Existiert nicht') AS Title,
    COALESCE(lo.PostURL, 'Existiert nicht') AS URL,
    COALESCE(lo.ContentType, 'Existiert nicht') AS PostType,



    -- Organisch
    COALESCE(lo.TotalShares, 0) AS Organic_TotalShares,
    COALESCE(lo.TotalComments, 0) AS Organic_TotalComments,
    COALESCE(lo.TotalLikes, 0) AS Organic_TotalLikes,
    COALESCE(lo.TotalClicks, 0) AS Organic_TotalClicks,
    COALESCE(lo.TotalImpressions, 0) AS Organic_TotalImpressions,
    COALESCE(lo.TotalEngagements, 0) AS Organic_TotalEngagements,
    COALESCE(ROUND(lo.TotalWeightedEngagements, 2), 0) AS Organic_TotalWeightedEngagements,
    COALESCE(lo.EngagementRateInPercent, 0) AS Organic_TotalEngagementRateInPercent,   


    -- Paid
    COALESCE(lp.ID, 'Existiert nicht') AS AdID,
    COALESCE(lp.CampaignName, 'Existiert nicht') AS CampaignName,
    COALESCE(lp.AdName, 'Existiert nicht') AS AdName,
    COALESCE(ROUND(lp.TotalSpend, 2), 0) AS TotalSpend,
    COALESCE(lp.TotalImpressions, 0) AS Paid_TotalImpressions,
    COALESCE(lp.TotalEngagements, 0) AS Paid_TotalEngagements,
    COALESCE(ROUND(lp.TotalWeightedEngagements, 2), 0) AS Paid_TotalWeightedEngagements,
    COALESCE(lp.EngagementRatings, 0) AS Paid_TotalEngagementRating,
    COALESCE(lp.TotalPostShares, 0) AS Paid_TotalShares,
    COALESCE(lp.TotalPostComments, 0) AS Paid_TotalComments,
    COALESCE(lp.TotalLikes, 0) AS Paid_TotalLikes,
    COALESCE(lp.TotalClicks, 0) AS Paid_TotalClicks,
    COALESCE(lp.CPM, 0) AS CPM,
    COALESCE(lp.CPC, 0) AS CPC,
    COALESCE(lp.CTR, 0) AS CTR,
    COALESCE(lp.CPV, 0) AS CPV,
    COALESCE(lp.VideoViewRate, 0) AS VideoViewRate,

       -- Gesamtergebnisse
    COALESCE(Organic_TotalImpressions + Paid_TotalImpressions, 0) AS All_TotalImpressions,
    COALESCE(Organic_TotalEngagements + Paid_TotalEngagements, 0) AS All_TotalEngagements,
    COALESCE(ROUND(Organic_TotalWeightedEngagements + Paid_TotalWeightedEngagements, 2), 0) AS All_TotalWeightedEngagements,
    COALESCE(Organic_TotalEngagementRateInPercent + Paid_TotalEngagementRating, 0) AS All_TotalEngagementRateInPercent,
    COALESCE(Organic_TotalShares + Paid_TotalShares, 0) AS All_TotalShares,
    COALESCE(Organic_TotalComments + Paid_TotalComments, 0) AS All_TotalComments,
    COALESCE(Organic_TotalLikes + Paid_TotalLikes, 0) AS All_TotalLikes,
    COALESCE(Organic_TotalClicks + Paid_TotalClicks, 0) AS All_TotalClicks,

    -- Neue Themen-Spalten aus linkedin_organic_total
    COALESCE(lo.Themenbereich1, 'Existiert nicht') AS Themenbereich1,
    COALESCE(lo.Themenbereich2, 'Existiert nicht') AS Themenbereich2,
    COALESCE(lo.Themenbereich3, 'Existiert nicht') AS Themenbereich3,
    COALESCE(lo.Themenbereich1_Conf, 0) AS Themenbereich1_Conf,
    COALESCE(lo.Themenbereich2_Conf, 0) AS Themenbereich2_Conf,
    COALESCE(lo.Themenbereich3_Conf, 0) AS Themenbereich3_Conf,
    COALESCE(lo.Strategie2030, 0.0) AS Strategie2030,
    COALESCE(lo.FinanzierungEnergiewende, 0.0) AS FinanzierungEnergiewende,
    COALESCE(lo.EMobilitaet, 0.0) AS EMobilitaet,
    COALESCE(lo.VernetzeEnergiewelt, 0.0) AS VernetzeEnergiewelt,
    COALESCE(lo.TransformationGasnetzeWasserstoff, 0.0) AS TransformationGasnetzeWasserstoff,
    COALESCE(lo.ErneuerbareEnergien, 0.0) AS ErneuerbareEnergien,
    COALESCE(lo.DisponibleErzeugung, 0.0) AS DisponibleErzeugung,
    COALESCE(lo.IntelligenteStromnetze, 0.0) AS IntelligenteStromnetze,
    COALESCE(lo.EnBWAlsArbeitgeberIn, 0.0) AS EnBWAlsArbeitgeberIn,
    COALESCE(lo.NachhaltigkeitCSRESG, 0.0) AS NachhaltigkeitCSRESG,
    COALESCE(lo.MarkeEnBW, 0.0) AS MarkeEnBW,
    COALESCE(lo.CLevelErwaehnungen, 'Existiert nicht') AS CLevelErwaehnungen
FROM datif_pz_uk_{env}.03_transformed.linkedin_organic_total AS lo
FULL OUTER JOIN datif_pz_uk_{env}.03_transformed.linkedin_paid_total AS lp
    ON TRIM(LOWER(lo.PostContent)) = TRIM(LOWER(lp.Text))
""")